# MRTrix Preprocessing Pipeline
- via: https://andysbrainbook.readthedocs.io/en/latest/MRtrix/MRtrix_Course/MRtrix_04_Preprocessing.html
- General approach in this notebook: 
    - build an initial dataframe containing the subject ids
    - apply preprocessing steps, where each step creates a new subdirectory and/or file
    - store the filepaths in new columns with

In [2]:
import numpy as np
import pandas as pd

from nipype.interfaces.dcm2nii import Dcm2nii

In [3]:
# specify project directory
main_dir = '/media/forest/wd_1/data_CLMS/diffusion'

## Prep Data for Preprocessing
- specified `home_dir` should contain subdirectories, 1 for each subject at one timepoint (m00 or m24)
- each subdirectory should contain files with extension `.mif`

### dcm to nifti+bval+bvec
-  dcm2nii: https://www.nitrc.org/plugins/mwiki/index.php/dcm2nii:MainPage
    - prereq: `sudo apt install mricron`
    - alternatively, can use the wrapper in nipype
        - https://nipype.readthedocs.io/en/latest/api/generated/nipype.interfaces.dcm2nii.html
- dcm files for each subject are stored inside the subdirectory: `1_dcm`


In [4]:
# specify subdirectory storing dcm files
dcm_dir = main_dir + '/1.1_dcm'

# create new dataframe to store file paths; subj id format: \w{3}_\d{3}_m\d{2}
subject_names = !ls $dcm_dir # dcm data should be stored in subdirectories named by subject
subject_dcm_dir = !ls -d $dcm_dir/* 

df_filepaths = pd.DataFrame({'subject_names': subject_names, 'dcm_path': subject_dcm_dir})
df_filepaths

,subject_names,dcm_path
0,aah_032_m00,/media/forest/wd_1/data_CLMS/diffusion/1.1_dcm...
1,aaj_034_m00,/media/forest/wd_1/data_CLMS/diffusion/1.1_dcm...
2,aao_039_m00,/media/forest/wd_1/data_CLMS/diffusion/1.1_dcm...
3,bbe_055_m00,/media/forest/wd_1/data_CLMS/diffusion/1.1_dcm...
4,ghi_007_m00,/media/forest/wd_1/data_CLMS/diffusion/1.1_dcm...
5,ijk_009_m00,/media/forest/wd_1/data_CLMS/diffusion/1.1_dcm...
6,mno_013_m00,/media/forest/wd_1/data_CLMS/diffusion/1.1_dcm...
7,pqr_016_m00,/media/forest/wd_1/data_CLMS/diffusion/1.1_dcm...
8,vwx_022_m00,/media/forest/wd_1/data_CLMS/diffusion/1.1_dcm...
9,xyz_024_m00,/media/forest/wd_1/data_CLMS/diffusion/1.1_dcm...


In [5]:
# Process dcm files into nifti

# check each row of the subject_dcm_dir column
# pass to dcm2nii to convert data to nifti
for subject, dcm in zip(df_filepaths['subject_names'], df_filepaths['dcm_path']):

    nifti_dir_exists = ![ -d "{main_dir}/1.2_nifti/{subject}" ] && echo 'True';
    if nifti_dir_exists == ['True']:
        print('Directory already exists!')
    else:
        !echo "Creating Subdirectory: {main_dir}/1.2_nifti/{subject}"
        !mkdir "{main_dir}/1.2_nifti/{subject}"
        !echo "Running dcm2nii..."
        # run command
        !dcm2nii -o {main_dir}/1.2_nifti/{subject} {dcm_dir}/{subject}

Directory already exists!
Directory already exists!
Directory already exists!
Directory already exists!
Directory already exists!
Directory already exists!
Directory already exists!
Directory already exists!
Directory already exists!
Directory already exists!


In [6]:
# rename nifti, bval, bvec files to subject name only, then add columns to the dataframe 
for subject in df_filepaths['subject_names']:
    print(f'Subject name: {subject}...')
    nifti_file = !ls {main_dir}/1.2_nifti/{subject}/*.nii.gz
    bval_filepath = !ls {main_dir}/1.2_nifti/{subject}/*.bval
    bvec_filepath = !ls {main_dir}/1.2_nifti/{subject}/*.bvec
    !mv {nifti_file[0]} {main_dir}/1.2_nifti/{subject}/{subject}_dwi.nii.gz
    !mv {bval_filepath[0]} {main_dir}/1.2_nifti/{subject}/{subject}_dwi.bval
    !mv {bvec_filepath[0]} {main_dir}/1.2_nifti/{subject}/{subject}_dwi.bvec
    print("Files renamed.")

Subject name: aah_032_m00...
mv: '/media/forest/wd_1/data_CLMS/diffusion/1.2_nifti/aah_032_m00/aah_032_m00_dwi.nii.gz' and '/media/forest/wd_1/data_CLMS/diffusion/1.2_nifti/aah_032_m00/aah_032_m00_dwi.nii.gz' are the same file
mv: '/media/forest/wd_1/data_CLMS/diffusion/1.2_nifti/aah_032_m00/aah_032_m00_dwi.bval' and '/media/forest/wd_1/data_CLMS/diffusion/1.2_nifti/aah_032_m00/aah_032_m00_dwi.bval' are the same file
mv: '/media/forest/wd_1/data_CLMS/diffusion/1.2_nifti/aah_032_m00/aah_032_m00_dwi.bvec' and '/media/forest/wd_1/data_CLMS/diffusion/1.2_nifti/aah_032_m00/aah_032_m00_dwi.bvec' are the same file
Files renamed.
Subject name: aaj_034_m00...
mv: '/media/forest/wd_1/data_CLMS/diffusion/1.2_nifti/aaj_034_m00/aaj_034_m00_dwi.nii.gz' and '/media/forest/wd_1/data_CLMS/diffusion/1.2_nifti/aaj_034_m00/aaj_034_m00_dwi.nii.gz' are the same file
mv: '/media/forest/wd_1/data_CLMS/diffusion/1.2_nifti/aaj_034_m00/aaj_034_m00_dwi.bval' and '/media/forest/wd_1/data_CLMS/diffusion/1.2_nifti/a

In [7]:
# append filepaths to dataframe
niftis, bvals, bvecs = [], [], []

for subject in df_filepaths['subject_names']:
    tmp1 = !ls {main_dir}/1.2_nifti/{subject}/*.nii.gz
    tmp2 = !ls {main_dir}/1.2_nifti/{subject}/*.bval
    tmp3 = !ls {main_dir}/1.2_nifti/{subject}/*.bvec

    niftis.append(tmp1[0])
    bvals.append(tmp2[0])
    bvecs.append(tmp3[0])

In [8]:
df_filepaths['1.2_niftis'] = niftis
df_filepaths['1.2_bvals'] = bvals
df_filepaths['1.2_bvecs'] = bvecs

### Convert files to .mif

In [9]:
for subject, nifti, bval, bvec in zip(df_filepaths['subject_names'], df_filepaths['1.2_niftis'], df_filepaths['1.2_bvals'], df_filepaths['1.2_bvecs']):
    print(f'Converting subject: {subject}...')

    mif_dir_exists = ![ -d "{main_dir}/1.3_mif/{subject}" ] && echo 'True';
    if mif_dir_exists == ['True']:
        print('Directory already exists!')
    else:
        !echo "Creating Subdirectory: {main_dir}/1.3_mif/{subject}"
        !mkdir "{main_dir}/1.3_mif/{subject}"
        !echo "Running nifti to mif conversion..."
        # run command
        !mrconvert {nifti} {main_dir}/1.3_mif/{subject}/{subject}_dwi.mif -fslgrad {bvec} {bval}

Converting subject: aah_032_m00...
Creating Subdirectory: /media/forest/wd_1/data_CLMS/diffusion/1.3_mif/aah_032_m00
Running nifti to mif conversion...
mrconvert: [100%] uncompressing image "/media/forest/wd_1/data_CLMS/diffusion/1.2_nifti/aah_032_m00/aah_032_m00_dwi.nii.gz"[0K
mrconvert: [100%] copying from "/media/for..._m00/aah_032_m00_dwi.nii.gz" to "/media/for...032_m00/aah_032_m00_dwi.mif"[0K
Converting subject: aaj_034_m00...
Creating Subdirectory: /media/forest/wd_1/data_CLMS/diffusion/1.3_mif/aaj_034_m00
Running nifti to mif conversion...
mrconvert: [100%] uncompressing image "/media/forest/wd_1/data_CLMS/diffusion/1.2_nifti/aaj_034_m00/aaj_034_m00_dwi.nii.gz"[0K
mrconvert: [100%] copying from "/media/for..._m00/aaj_034_m00_dwi.nii.gz" to "/media/for...034_m00/aaj_034_m00_dwi.mif"[0K
Converting subject: aao_039_m00...
Creating Subdirectory: /media/forest/wd_1/data_CLMS/diffusion/1.3_mif/aao_039_m00
Running nifti to mif conversion...
mrconvert: [100%] uncompressing image "/medi

In [15]:
# append .mif files to the dataframe
mifs = []

for subject in df_filepaths['subject_names']:
    tmp = !ls {main_dir}/1.3_mif/{subject}/*.mif
    mifs.append(tmp)

df_filepaths['1.3_mif'] = mifs

### Validation & QC

In [36]:
# Ensure number of volumes matches with bval/bvec for all subjects
for subject, mif, bval, bvec in zip(df_filepaths['subject_names'], df_filepaths['1.3_mif'], df_filepaths['1.2_bvals'], df_filepaths['1.2_bvecs']):
    print(f'Validation for {subject}...')
    !echo 'Number of volumes in mif:'
    !mrinfo -size {mif[0]} | awk '{{print $$4}}'
    !echo 'Number of columns in bval:'
    !awk '{{print NF; exit}}' {bval}
    !echo 'Number of columns in bvec:'
    !awk '{{print NF; exit}}' {bvec}
    print()

    # write if statement to print out error indication if they don't match


Validation for aah_032_m00...
Number of volumes in mif:
140
Number of columns in bval:
140
Number of columns in bvec:
140

Validation for aaj_034_m00...
Number of volumes in mif:
140
Number of columns in bval:
140
Number of columns in bvec:
140

Validation for aao_039_m00...
Number of volumes in mif:
140
Number of columns in bval:
140
Number of columns in bvec:
140

Validation for bbe_055_m00...
Number of volumes in mif:
140
Number of columns in bval:
140
Number of columns in bvec:
140

Validation for ghi_007_m00...
Number of volumes in mif:
140
Number of columns in bval:
140
Number of columns in bvec:
140

Validation for ijk_009_m00...
Number of volumes in mif:
140
Number of columns in bval:
140
Number of columns in bvec:
140

Validation for mno_013_m00...
Number of volumes in mif:
140
Number of columns in bval:
140
Number of columns in bvec:
140

Validation for pqr_016_m00...
Number of volumes in mif:
140
Number of columns in bval:
140
Number of columns in bvec:
140

Validation for v

In [ ]:
# visual inspection--load data using ipywidgets & matplotlib

## Preprocessing

### Denoising

### Remove Gibbs Artifacts

In [ ]:
### Final 